3D Animation of reconstruction data
=======

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch as torch
import torch.nn as nn
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

Data

In [2]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
print(torch.cuda.get_device_name(0))

lorenz_train=np.load("../data/lorenz_train.npy")
lorenz_test=np.load("../data/lorenz_test.npy")
print("Train data size: ",lorenz_train.shape)
print("Test data size: ",lorenz_test.shape)

def data_preperation(data):
    t=np.arange(0,len(data[0,:,0]))
    X_train=torch.from_numpy(data[:,0:-1,:]).float().to(device)
    Y_train=torch.from_numpy(data[:,1:,:]).float().to(device)
    return X_train,Y_train,t

X_train,Y_train,t = data_preperation(lorenz_train)
X_test,Y_test,t = data_preperation(lorenz_test)

Using cuda device
NVIDIA GeForce GTX 1070
Train data size:  (100, 1000, 3)
Test data size:  (100, 1000, 3)


Hyperparameter

In [3]:
hidden_size = 20  # number of hidden units x RNN
input_size = len(lorenz_train[0,0,:])  # number of input units
output_size = len(lorenz_train[0,0,:])  # number of output units
num_layers = 1  # number of layers
batch_size = len(lorenz_train[:,0,0])  # size of the input data used for one iteration
criterion = nn.MSELoss()

LSTM model

In [7]:
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(MyRNN, self).__init__()  # Inherited from the parent class nn.Module
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)
        self.linear = nn.Linear(self.hidden_size, self.output_size)  # Define the output layer

    def forward(self, x):  # Forward pass: stacking each layer together

        output, hidden = self.lstm(x)
        output = self.linear(output)

        return output

Load model

In [8]:
model=torch.load('../model/LSTM_model_lorenz.pt')

Animation
------

In [9]:
# Calculate Prediction
y_pred = model(X_test)  # apply the trained model to training set

loss=criterion(y_pred,Y_test).item()
print("Loss: {:.8f}".format(round(loss,8)))

y_pred = y_pred.cpu().detach().numpy()
y = Y_test.cpu().detach().numpy()

Loss: 0.00006573


In [ ]:
# Create a figure and a 3D Axes
fig = plt.figure(figsize=(8,8))
ax = Axes3D(fig)

def init():
    # Plot the surface.
    ax.plot(y[0,:,0],y[0,:,1],y[0,:,2],label="true trajectory",color="royalblue",linewidth=2)
    ax.plot(y_pred[0,:,0],y_pred[0,:,1],y_pred[0,:,2],label="predicted trajectory",color="orangered",linewidth=1)
    ax.set_title("Lorenz Attractor")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    ax.legend()
    return fig,

def animate(i):
    # azimuth angle : 0 deg to 360 deg
    ax.view_init(elev=20, azim=i*4)
    return fig,

# Animate
ani = animation.FuncAnimation(fig, animate, init_func=init,frames=90, interval=100, blit=True)
plt.close()
HTML(ani.to_jshtml())

In [8]:
writergif = animation.PillowWriter(fps=10)
ani.save('filename.gif',writer=writergif)